In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer 
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
from numpy import nan
from bs4 import BeautifulSoup    
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from math import sqrt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
url_train = 'https://raw.githubusercontent.com/harsh-77/movie_rating/master/movie_train.tsv'
url_test = 'https://raw.githubusercontent.com/harsh-77/movie_rating/master/movie_test.tsv'
train_data = pd.read_csv(url_train,sep='\t')
test_data = pd.read_csv(url_test,sep='\t')

In [4]:
train_data.head(1)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1


In [5]:
test_data.head(1)

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...


In [6]:
Sentiment_words=[]
for row in train_data['Sentiment']:
    if row ==0:
        Sentiment_words.append('negative')
    elif row == 1:
        Sentiment_words.append('neutral')
    elif row == 2:
        Sentiment_words.append('somewhat negative')
    elif row == 3:
        Sentiment_words.append('somewhat positive')
    elif row == 4:
        Sentiment_words.append('positive')
    else:
        Sentiment_words.append('Failed')
train_data['Sentiment_words'] = Sentiment_words

In [7]:
def review_to_words(raw_review): 
    review =raw_review
    review = re.sub('[^a-zA-Z]', ' ',review)
    review = review.lower()
    review = review.split()
    lemmatizer = WordNetLemmatizer()
    review = [lemmatizer.lemmatize(w) for w in review if not w in set(stopwords.words('english'))]
    return (' '.join(review))


In [8]:
corpus= []
for i in range(0, 156060):
    corpus.append(review_to_words(train_data['Phrase'][i]))

In [9]:
train_data['new_Phrase']=corpus
train_data.drop(['Phrase'],axis=1,inplace=True)
train_data.head()

,PhraseId,SentenceId,Sentiment,Sentiment_words,new_Phrase
0,1,1,1,neutral,series escapade demonstrating adage good goose...
1,2,1,2,somewhat negative,series escapade demonstrating adage good goose
2,3,1,2,somewhat negative,series
3,4,1,2,somewhat negative,
4,5,1,2,somewhat negative,series


In [10]:
positive=train_data[train_data['Sentiment_words']==('positive')]
words = ' '.join(positive['new_Phrase'])
split_word = " ".join([word for word in words.split()])

In [11]:
wordcloud = WordCloud(stopwords=STOPWORDS,
                      background_color='black',
                      width=3000,
                      height=2500
                     ).generate(split_word)

In [12]:
pos=positive['new_Phrase']

In [13]:
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 2000) 

In [14]:
pos_words = vectorizer.fit_transform(pos)
pos_words = pos_words.toarray()
pos= vectorizer.get_feature_names()

In [18]:
pos[1]

'able'

In [19]:
print (pos_words.shape)

(9206, 2000)


In [20]:
dist = np.sum(pos_words, axis=0)

In [21]:
postive_new= pd.DataFrame(dist)

In [22]:
postive_new.columns=['word_count']
postive_new['word'] = pd.Series(pos, index=postive_new.index)
postive_new1=postive_new[['word','word_count']]

In [24]:
postive_new1.head(2)

,word,word_count
0,ability,20
1,able,32


In [25]:
corpus1= []
for i in range(0, 66291):
    corpus1.append(review_to_words(test_data['Phrase'][i]))

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
x__train = cv.fit_transform(corpus).toarray()
x__test= cv.fit_transform(corpus1).toarray()
y = train_data.iloc[:, 2].values

In [28]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x__train, y, test_size = 0.40, random_state = 0)

from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
y_real_pred = classifier.predict(x__test)

In [47]:
y_pred.shape


(62424,)

In [48]:
y_real_pred.shape

(66291,)

In [46]:
a= np.array(train_data['Sentiment'])
a.shape

(156060,)

In [49]:
#from sklearn import metrics
#metrics.accuracy_score(y_pred, a)

In [39]:
np.array(train_data['Sentiment'])

array([1, 2, 2, ..., 3, 2, 2], dtype=int64)

In [53]:
subm= pd.DataFrame({'Sentiment': y_real_pred })
subm.to_csv('movie.csv', index= True)

In [54]:
mse = ((y_pred - y_test) ** 2).mean()

In [55]:
mse

0.6473792131231577

In [56]:
rmse = sqrt(mse)

In [57]:
rmse

0.8045987901576522